# IMDB 영화평 감성분석(이진 분류)

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('data\labeledTrainData.tsv\labeledTrainData.tsv', sep='\t')
df.head(3)

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...


In [4]:
df = pd.read_csv('data\labeledTrainData.tsv\labeledTrainData.tsv', sep='\t', quoting=3) # 3: QUOTE-None
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [6]:
df.review[0][:1000]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

- 텍스트 전처리

In [7]:
# <br /> 태그는 공백으로 변환
df.review = df.review.str.replace('<br />',' ')
df.review

0        "With all this stuff going down at the moment ...
1        "\"The Classic War of the Worlds\" by Timothy ...
2        "The film starts with a manager (Nicholas Bell...
3        "It must be assumed that those who praised thi...
4        "Superbly trashy and wondrously unpretentious ...
                               ...                        
24995    "It seems like more consideration has gone int...
24996    "I don't believe they made this film. Complete...
24997    "Guy is a loser. Can't get girls, needs to bui...
24998    "This 30 minute documentary Buñuel made in the...
24999    "I saw this movie as a child and it broke my h...
Name: review, Length: 25000, dtype: object

In [8]:
a = 'nice of him.<br /><br />The actual feature'
a.replace('<br />', ' ')

'nice of him.  The actual feature'

In [9]:
# 구둣점, 숫자 제거 - 영문자가 아닌 글자는 공백으로 변환, 양쪽에 있는 blank 제거
# DF의 문자열 조작할 때는, str.함수명()을 붙여야 한다. => DF에서 문자열 메소드 사용할 때는 기억해 두기
# Series에는 .replace() attribute를 제공하지 않다고 에러가 뜬다.
df.review = df.review.str.replace('[^A-Za-z]', ' ').str.strip()
df.review[0][:1000]

<ipython-input-9-1ea54e03b925>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df.review = df.review.str.replace('[^A-Za-z]', ' ').str.strip()


'With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for  

- Train/Test dataset 분리

In [10]:
#  X_train, y_train은 Series이기 때문에, Series명.values_counts()를 사용한다. 
# 비교) numpy형식일 경우에는 np.unique() 메소드를 사용

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.review, df.sentiment, stratify=df.sentiment, random_state=2022
)

y_train.value_counts()

0    9375
1    9375
Name: sentiment, dtype: int64

In [11]:
 y_test.value_counts()

0    3125
1    3125
Name: sentiment, dtype: int64

- CountVectorizer로 변환

- 변환: fit_transform()
- X_train, X_test => fit_transform()해야 한다면?
    - X_train : X_test = 3:1이라 가정해보자
    - unique한 단어의 개수가 X_train = 30,000 , X_test = 10,000이라고 봐도 무방할까?
        - CountVectorizer -> X_train은 0~29999의 index, X_test는 0~9999 index
        - X_train에서는 at이라는 단어가 100번째, X_test에서는 50번째가 될 수 있다. => train과 test가 불일치! 
        - 어떻게 해결해야 할까?
            - X_train 학습(fit) -> transform
            - **X_test는 X_train 학습한 데이터로 학습해야 한다**

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(stop_words='english')

In [13]:
# Line이 18750개, 단어의 개수가 65213
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_train_cv.shape

(18750, 65213)

In [14]:
# 잘못된 방법
cvect2 = CountVectorizer(stop_words='english')
X_test_cv = cvect2.fit_transform(X_test)
X_test_cv.shape

(6250, 41343)

In [15]:
# 훈련 데이터와 테스트 데이터가 다르게 나옴
def get_word(index, voca):
    for key, value in voca.items():
        if value == index:
            return key

get_word(100, cvect.vocabulary_), get_word(100, cvect2.vocabulary_)

('abilityof', 'abos')

In [16]:
# 훈련 데이터와 Line과 단어의 개수가 같다.
# test dataset은 train dataset을 변환한 Vectorizer로 반드시 변환하여야 함
X_test_cv = cvect.transform(X_test)
X_test_cv.shape

(6250, 65213)

### 분류기 - LogisticRegression

In [19]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=2022, max_iter=500)
lr.fit(X_train_cv, y_train)

LogisticRegression(max_iter=500, random_state=2022)

In [20]:
lr.score(X_test_cv, y_test)

0.86576

- CountVectorizer + ngram_range=(1,2)

In [33]:
cvect12 = CountVectorizer(stop_words='english', ngram_range=(1,2))

In [34]:
cvect12.fit(X_train)
%time X_train_cv12 = cvect12.transform(X_train)

Wall time: 4.5 s


In [35]:
X_test_cv12 = cvect12.transform(X_test)
X_train_cv12.shape, X_test_cv12.shape

((18750, 1386558), (6250, 1386558))

In [36]:
lr12 = LogisticRegression(max_iter=500, random_state=2022)
lr12.fit(X_train_cv12, y_train)
lr12.score(X_test_cv12, y_test)

0.87968

- 모델 저장하고 불러오기

In [39]:
import joblib

In [40]:
# 모델 저장
joblib.dump(cvect12, 'model/imdb_cvect12.pkl')
joblib.dump(lr12, 'model/imdb_cvect_lr12.pkl')

['model/imdb_cvect_lr12.pkl']

In [41]:
del cvect12, lr12

In [42]:
# 모델 로딩
cvect12 = joblib.load('model/imdb_cvect12.pkl')
lr12 = joblib.load('model/imdb_cvect_lr12.pkl')

- 실제 데이터로 검증

In [44]:
review = '''
\nWhat cinemas were made for. I wasn't expecting something quite as amazing as this, this was two and a half hours of incredible entertainment, drama, laughs, tears and action galore, there truly was something for everyone here.\n\nThis movie really has ignited my love for the franchise once again, and I am truly excited for what comes next.\n\nNostalgia, there seems to be a real thirst for it theses days, and there is a lot of it to enjoy here. There are plenty of moments throughout that will have you speechless, they really do break all of the rules here.\n\nThis is one incredibly good looking movie, dazzling special effects, dizzying action scenes, two and a half hours will literally fly by.\n\nIncredible, 10/10.\n
'''

review

"\n\nWhat cinemas were made for. I wasn't expecting something quite as amazing as this, this was two and a half hours of incredible entertainment, drama, laughs, tears and action galore, there truly was something for everyone here.\n\nThis movie really has ignited my love for the franchise once again, and I am truly excited for what comes next.\n\nNostalgia, there seems to be a real thirst for it theses days, and there is a lot of it to enjoy here. There are plenty of moments throughout that will have you speechless, they really do break all of the rules here.\n\nThis is one incredibly good looking movie, dazzling special effects, dizzying action scenes, two and a half hours will literally fly by.\n\nIncredible, 10/10.\n\n"

In [48]:
# string replace methos는 정규표현식을 지원하지 않음
review.replace('[^A-Za-z]' , ' ')

"\n\nWhat cinemas were made for. I wasn't expecting something quite as amazing as this, this was two and a half hours of incredible entertainment, drama, laughs, tears and action galore, there truly was something for everyone here.\n\nThis movie really has ignited my love for the franchise once again, and I am truly excited for what comes next.\n\nNostalgia, there seems to be a real thirst for it theses days, and there is a lot of it to enjoy here. There are plenty of moments throughout that will have you speechless, they really do break all of the rules here.\n\nThis is one incredibly good looking movie, dazzling special effects, dizzying action scenes, two and a half hours will literally fly by.\n\nIncredible, 10/10.\n\n"

In [45]:
# 텍스트 전처리 - 영문자 이외는 공백처리
import re                   # regular expression(정규 표현식)
clean_review = re.sub('[^A-Za-z]', ' ', review).strip()

In [46]:
clean_review

'What cinemas were made for  I wasn t expecting something quite as amazing as this  this was two and a half hours of incredible entertainment  drama  laughs  tears and action galore  there truly was something for everyone here   This movie really has ignited my love for the franchise once again  and I am truly excited for what comes next   Nostalgia  there seems to be a real thirst for it theses days  and there is a lot of it to enjoy here  There are plenty of moments throughout that will have you speechless  they really do break all of the rules here   This is one incredibly good looking movie  dazzling special effects  dizzying action scenes  two and a half hours will literally fly by   Incredible'

In [49]:
review_cv = cvect12.transform([clean_review])

In [51]:
review_cv.shape

(1, 1386558)

In [52]:
lr12.predict(review_cv)

array([1], dtype=int64)